In [37]:
from qiskit import transpile, schedule, assemble
import pickle
from qiskit import pulse, execute, QuantumCircuit
from qiskit.pulse.library import GaussianSquare
from qiskit.pulse import Schedule, Play, Acquire
import numpy as np
import typing
import matplotlib.pyplot as plt
from qiskit.pulse.channels import DriveChannel, ControlChannel, AcquireChannel, MeasureChannel, MemorySlot
from qiskit import IBMQ
from qiskit.pulse.instructions.play import Play
from qiskit.transpiler.passes.scheduling.calibration_creators import RZXCalibrationBuilder

In [38]:
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter, TomographyFitter
from qiskit.ignis.verification import marginal_counts
from qiskit.quantum_info.operators.channel import Choi, Kraus, PTM
from qiskit.circuit import QuantumCircuit, QuantumRegister, Gate
from qiskit import transpile, schedule, assemble
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.visualization import(
  plot_state_city,
  plot_bloch_multivector,
  plot_state_paulivec,
  plot_state_hinton,
  plot_state_qsphere)

In [39]:
from qiskit.providers.aer.noise.errors.errorutils import standard_gate_unitary
from qiskit.quantum_info.operators.measures import process_fidelity
from qiskit.quantum_info import Choi, Kraus, Operator
import qiskit.quantum_info as qi

In [4]:
provider = IBMQ.load_account()

In [5]:
provider_pulse =IBMQ.get_provider(hub='ibm-q-startup', group='phasecraft-ltd', project='pulse-level-cont')

In [40]:
backend = provider_pulse.get_backend('ibmq_bogota')#,mode='dedicated')

config = backend.configuration()
props = backend.properties()
defaults = backend.defaults()
calibrations = defaults.instruction_schedule_map

In [ ]:
RZX_cals = RZXCalibrationBuilder(backend)

In [ ]:
qubits = (1,2) #picking ones with longest gate times to see if stronger effect
angles = np.round(np.linspace(0.1, np.pi, 20), 2)

In [ ]:
schedules = [RZX_cals.get_calibration([angle] , qubits) for angle in angles] #refresh every calibration

In [ ]:
for angle, sched in zip(angles, schedules):
    gate_name =f'rzx_{angle}'
    basis_gates += [gate_name]
    inst_map.add(gate_name, qubits, sched)

In [ ]:
gates = [Gate(f'rzx_{angle}', 2, []) for angle in angles]

In [ ]:
label = '1.May.bogota'
results = {}
qregs = QuantumRegister(config.n_qubits)

for angle, gate in zip(angles, gates):
    circuit = QuantumCircuit(qregs)
    circuit.append(gate, qargs=[qregs[qubits[0]], qregs[qubits[1]]]) 
    qpt_circuits = process_tomography_circuits(circuit, [qregs[qubits[0]], qregs[qubits[1]]]) 
    qpt_circuits = transpile(qpt_circuits, backend, basis_gates)
    qpt_schedules = schedule(qpt_circuits, backend, inst_map)
    job = execute(qpt_schedules, backend=backend, shots=512)
    result = job.result()
    results[f'{angle}'] = {'result': result, 'circuit': circuit, 'qpt_scheds':qpt_schedules, 'qpt_circuits':qpt_circuits}

with open(f'results.{label}.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
fidelities = {}
q2 = QuantumRegister(2)

for angle, gate in zip(angles, gates):
   
    qpt_circuits = results[str(angle)]['qpt_circuits']
    res = results[str(angle)]['result']
    
    process_data = ProcessTomographyFitter(res, qpt_circuits,meas_basis='Pauli', prep_basis='Pauli')
    choi_fit = process_data.fit( method='cvx', standard_weights=True,kwargs={ "trace_preserving":True, "PSD": True})
    choi_fit = choi_fit.data
   
    ideal_circ = QuantumCircuit(q2)
    ideal_circ.rzx(angle,q2[0],q2[1])
    ideal=qi.Operator(ideal_circ)
    fid = qi.average_gate_fidelity(channel=Choi(choi_fit),target=Operator(ideal.data))
    fidelities[str(angle)] = fid